In [14]:
import os
import json

def find_best_config_str(exp_path, mode='retrieval', score_by='MAP'):
    l = []
    # find al files called log.txt, recursively
    for root, dirs, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # open file, and for each line test if it contains the string "result for verification run - "
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'result for {mode} run' in line:
                            score = float(line.split(f'{score_by}: ')[1].split(' ')[0])
                            # now, get string between the brackets { and }
                            configs = '{' + line.split('{')[1].split('}')[0] + '}'
                            # replace single quotes with double quotes
                            configs = configs.replace("'", '"')
                            # convert each boolean value to string
                            configs = configs.replace(': True,', ': "True",')
                            configs = configs.replace(': False,', ': "False",')
                            
                            # convert to json
                            config = json.loads(configs)
                            l.append((score, config))

    # sort the list by the first element, which is the score
    l.sort(key=lambda x: x[0], reverse=True)
    return l

In [34]:
def find_best_config_fp(exp_path, mode='retrieval', score_by='MAP'):
    l = []
    # find al files called log.txt, recursively
    for root, dirs, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # open file, and for each line test if it contains the string "result for verification run - "
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'result for {mode} run' in line:
                            score = float(line.split(f'{score_by}: ')[1].split(' ')[0])
                            config = json.load(open(os.path.join(f.name, '..', '..', 'config.json')))
                            l.append((score, config))

    # sort the list by the first element, which is the score
    l.sort(key=lambda x: x[0], reverse=True)
    return l

exp_path = 'data-out/experiments/test-all/'

mode = 'verification'
score_by='F1'
configs = find_best_config_fp(exp_path, mode, score_by)

FileNotFoundError: [Errno 2] No such file or directory: 'data-out/experiments/test-all/retr-OPENAI\\nopre-name-bio\\nonorm-noscale-noignore_nei\\eval\\log.txt\\..\\..\\config.json'

In [35]:
exp_path = 'data-out/experiments/test-all/'

# # print the top 10 results
# mode = 'retrieval'
# score_by='MAP'
# configs = find_best_config(exp_path, mode, score_by)
# for i, (f1, config) in enumerate(configs[:2]):
#     print(f'{i}. {score_by} score: {f1:.4f}, config: {config}')

# print the top 10 results
mode = 'verification'
score_by='F1'
configs = find_best_config_fp(exp_path, mode, score_by)
for i, (score, config) in enumerate(configs[:]):
    print(f'{i}. {score_by} score: {score:.4f}, config: {config}')


FileNotFoundError: [Errno 2] No such file or directory: 'data-out/experiments/test-all/retr-OPENAI\\nopre-name-bio\\nonorm-noscale-ignore_nei\\eval\\log.txt\\..\\..\\config.json'